# 常规赛：PALM病理性近视病灶检测与分割

## 问题分析

一个二分割任务，只是同样的数据要做两种不同的分割模型。本程序使用PaddleSeg进行分割。

### 特别注意

Paddleseg中训练时以255作为忽略像素点

## 数据准备

In [ ]:
# 下载数据与解压数据
import os
import urllib

url='https://bj.bcebos.com/v1/dataset-bj/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/%E5%B8%B8%E8%A7%84%E8%B5%9B%EF%BC%9APALM%E7%97%85%E7%90%86%E6%80%A7%E8%BF%91%E8%A7%86%E7%97%85%E7%81%B6%E6%A3%80%E6%B5%8B%E4%B8%8E%E5%88%86%E5%89%B2.zip'

if not os.path.exists('./work/Train_and_test.zip'):
    print("Downloading start!")
    urllib.request.urlretrieve(url, "./work/Train_and_test.zip")  
    print("Downloading end!")
else:
    print("Already Downloading")

! unzip -oq ./work/Train_and_test.zip -d ./work

## 导入包

In [1]:
!pip install paddleseg

#导入常用的库
import os
import random
import numpy as np
from random import shuffle
import cv2
import paddle
from PIL import Image
import shutil
import re
from paddle.vision.transforms import functional as F
import os.path
import paddleseg.transforms as T
from paddleseg.datasets import OpticDiscSeg,Dataset
import paddleseg.models
from paddleseg.models import UNet
from paddleseg.models import OCRNet
from paddleseg.models.losses import CrossEntropyLoss,DiceLoss, MixedLoss
from paddleseg.core import train
from paddleseg.core import evaluate
from paddleseg.core import predict
from PIL import Image

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 245kB 9.6MB/s eta 0:00:01


## Atr分割部分
### 数据准备

In [5]:
! mkdir /home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Atrophy_png

names=[item.split('.')[0] for item in os.listdir('work/常规赛：PALM病理性近视病灶检测与分割/Train/fundus_image')]
ori_train_list=[]
img_dir='/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/fundus_image/'
mask_dir='/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Atrophy/'
mask_dir2='/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Atrophy_png/'
for name in names:
    if os.path.exists(img_dir+name+'.jpg') and os.path.exists(mask_dir+name+'.bmp'):
        im=Image.open(mask_dir+name+'.bmp')
        im=(np.array(im)/255).astype('uint8')
    else: #if os.path.exists(img_dir+name+'.jpg') and ! os.path.exists(mask_dir+name+'.bmp'):
        im=Image.open(img_dir+name+'.jpg')
        row=np.array(im).shape[0]
        col=np.array(im).shape[1]
        im=np.ones([row,col],dtype='int64').astype('uint8')
    im = Image.fromarray(im)
    im.save(mask_dir2+name+'.png')
    ori_train_list.append(img_dir+name+'.jpg '+mask_dir2+name+'.png'+'\n')

mkdir: cannot create directory ‘/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Atrophy_png’: File exists


In [6]:
# 分割验证集
cut_percent=0.2
cut_point=int(cut_percent*len(ori_train_list))
random.shuffle(ori_train_list)

with open('atr_train.txt','w') as f:
    for item in ori_train_list[cut_point:]:
        f.write(item)

with open('atr_eval.txt','w') as f:
    for item in ori_train_list[:cut_point]:
        f.write(item)

### 训练

In [7]:
train_transforms = [
    #T.RandomHorizontalFlip(),# 水平翻转
    #T.RandomVerticalFlip(),# 垂直翻转
    #T.RandomDistort(0.4),# 随机扭曲
    #T.RandomBlur(0.3),# 高斯模糊
    #T.RandomScaleAspect(min_scale=0.5,aspect_ratio=0.5),# 随机缩放
    T.Resize(target_size=(512,512)),
    T.Normalize()  # 图像标准化
]

val_transforms = [
    T.Resize(target_size=(512,512)),
    T.Normalize()
]

In [8]:
# 构建训练集
train_dataset = Dataset(
    dataset_root='/home/aistudio',
    train_path='/home/aistudio/atr_train.txt',
    transforms=train_transforms,
    num_classes=2,
    mode='train'
)
#验证集
val_dataset = Dataset(
    dataset_root='/home/aistudio',
    val_path='/home/aistudio/atr_eval.txt',
    transforms=val_transforms,
    num_classes=2,
    mode='val'
)

In [9]:
# unet_model = UNet(num_classes=2,pretrained='/home/aistudio/output/PLAM_model_3/best_model/model.pdparams')#使用预训练模型unet进行训练
model = UNet(num_classes=2)#使用unet进行训练

In [11]:
base_lr =0.001
iters = 1000

#自动调整学习率
lr =paddle.optimizer.lr.CosineAnnealingDecay(base_lr, T_max=(iters // 3), last_epoch=0.5)
u_optimizer = paddle.optimizer.Adam(lr, parameters=model.parameters())

mixtureLosses = [CrossEntropyLoss(),DiceLoss() ]
mixtureCoef = [0.6,0.4]
losses = {}
losses['types'] = [MixedLoss(mixtureLosses, mixtureCoef)]
losses['coef'] = [1]

In [12]:
#进行训练
train(
    model = model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=u_optimizer,
    save_dir='output/Atr_UNet',
    iters=iters,  
    batch_size=8, 
    save_interval=200,
    log_iters=20,
    num_workers=2,
    losses=losses,
    # losses=CrossEntropyLoss(),
    use_vdl=False
    )

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.int64, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


2021-09-25 16:06:53 [INFO]	[TRAIN] epoch: 1, iter: 20/1000, loss: 0.3189, lr: 0.000991, batch_cost: 0.7416, reader_cost: 0.25537, ips: 10.7881 samples/sec | ETA 00:12:06
2021-09-25 16:07:06 [INFO]	[TRAIN] epoch: 1, iter: 40/1000, loss: 0.2371, lr: 0.000964, batch_cost: 0.6598, reader_cost: 0.17488, ips: 12.1248 samples/sec | ETA 00:10:33
2021-09-25 16:07:19 [INFO]	[TRAIN] epoch: 1, iter: 60/1000, loss: 0.2375, lr: 0.000921, batch_cost: 0.6605, reader_cost: 0.17665, ips: 12.1112 samples/sec | ETA 00:10:20
2021-09-25 16:07:32 [INFO]	[TRAIN] epoch: 1, iter: 80/1000, loss: 0.2174, lr: 0.000863, batch_cost: 0.6622, reader_cost: 0.17883, ips: 12.0813 samples/sec | ETA 00:10:09
2021-09-25 16:07:46 [INFO]	[TRAIN] epoch: 2, iter: 100/1000, loss: 0.2013, lr: 0.000792, batch_cost: 0.6964, reader_cost: 0.21270, ips: 11.4873 samples/sec | ETA 00:10:26
2021-09-25 16:07:59 [INFO]	[TRAIN] epoch: 2, iter: 120/1000, loss: 0.2129, lr: 0.000710, batch_cost: 0.6351, reader_cost: 0.15084, ips: 12.5963 sampl

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int32, but right dtype is paddle.bool, the right dtype will convert to paddle.int32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.bool, the right dtype will convert to paddle.int64
  format(lhs_dtype, rhs_dtype, lhs_dtype))
160/160 [==============================] - 26s 162ms/step - batch_cost: 0.1611 - reader cost: 0.14


2021-09-25 16:09:18 [INFO]	[EVAL] #Images: 160 mIoU: 0.6848 Acc: 0.9560 Kappa: 0.5658 
2021-09-25 16:09:18 [INFO]	[EVAL] Class IoU: 
[0.4149 0.9546]
2021-09-25 16:09:18 [INFO]	[EVAL] Class Acc: 
[0.8656 0.9594]
2021-09-25 16:09:19 [INFO]	[EVAL] The model with the best validation mIoU (0.6848) was saved at iter 200.
2021-09-25 16:09:30 [INFO]	[TRAIN] epoch: 3, iter: 220/1000, loss: 0.1697, lr: 0.000256, batch_cost: 0.5635, reader_cost: 0.07718, ips: 14.1960 samples/sec | ETA 00:07:19
2021-09-25 16:09:43 [INFO]	[TRAIN] epoch: 3, iter: 240/1000, loss: 0.1808, lr: 0.000179, batch_cost: 0.6226, reader_cost: 0.13678, ips: 12.8489 samples/sec | ETA 00:07:53
2021-09-25 16:09:57 [INFO]	[TRAIN] epoch: 4, iter: 260/1000, loss: 0.1919, lr: 0.000112, batch_cost: 0.6972, reader_cost: 0.20993, ips: 11.4751 samples/sec | ETA 00:08:35
2021-09-25 16:10:10 [INFO]	[TRAIN] epoch: 4, iter: 280/1000, loss: 0.1635, lr: 0.000060, batch_cost: 0.6671, reader_cost: 0.18090, ips: 11.9920 samples/sec | ETA 00:08:00

160/160 [==============================] - 25s 155ms/step - batch_cost: 0.1532 - reader cost: 0.137


...

160/160 [==============================] - 25s 153ms/step - batch_cost: 0.1520 - reader cost: 0.13


2021-09-25 16:17:03 [INFO]	[EVAL] #Images: 160 mIoU: 0.7917 Acc: 0.9698 Kappa: 0.7458 
2021-09-25 16:17:03 [INFO]	[EVAL] Class IoU: 
[0.6152 0.9682]
2021-09-25 16:17:03 [INFO]	[EVAL] Class Acc: 
[0.8537 0.9767]
2021-09-25 16:17:04 [INFO]	[EVAL] The model with the best validation mIoU (0.7917) was saved at iter 800.
2021-09-25 16:17:17 [INFO]	[TRAIN] epoch: 11, iter: 820/1000, loss: 0.1649, lr: 0.000556, batch_cost: 0.6845, reader_cost: 0.19659, ips: 11.6867 samples/sec | ETA 00:02:03
2021-09-25 16:17:30 [INFO]	[TRAIN] epoch: 11, iter: 840/1000, loss: 0.1376, lr: 0.000462, batch_cost: 0.6301, reader_cost: 0.14171, ips: 12.6955 samples/sec | ETA 00:01:40
2021-09-25 16:17:43 [INFO]	[TRAIN] epoch: 11, iter: 860/1000, loss: 0.1164, lr: 0.000369, batch_cost: 0.6302, reader_cost: 0.14310, ips: 12.6950 samples/sec | ETA 00:01:28
2021-09-25 16:17:55 [INFO]	[TRAIN] epoch: 11, iter: 880/1000, loss: 0.1419, lr: 0.000281, batch_cost: 0.6283, reader_cost: 0.13986, ips: 12.7323 samples/sec | ETA 00:0

160/160 [==============================] - 24s 149ms/step - batch_cost: 0.1477 - reader cost: 0.132


2021-09-25 16:19:37 [INFO]	[EVAL] #Images: 160 mIoU: 0.8163 Acc: 0.9726 Kappa: 0.7817 
2021-09-25 16:19:37 [INFO]	[EVAL] Class IoU: 
[0.6617 0.971 ]
2021-09-25 16:19:37 [INFO]	[EVAL] Class Acc: 
[0.8329 0.9822]
2021-09-25 16:19:38 [INFO]	[EVAL] The model with the best validation mIoU (0.8163) was saved at iter 1000.
<class 'paddle.nn.layer.conv.Conv2D'>'s flops has been counted
<class 'paddle.nn.layer.norm.BatchNorm2D'>'s flops has been counted
Cannot find suitable count function for <class 'paddle.nn.layer.pooling.MaxPool2D'>. Treat it as zero FLOPs.
Total Flops: -1150812160     Total Params: 13404354


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


### 进行预测，保存结果到指定目录

In [14]:
test_list=["work/常规赛：PALM病理性近视病灶检测与分割/PALM-Testing400-Images/"+item for item in os.listdir("work/常规赛：PALM病理性近视病灶检测与分割/PALM-Testing400-Images")]

transforms = T.Compose([
    T.Resize(target_size=(512, 512)),
    T.Normalize()
])

model = paddleseg.models.UNet(num_classes=2)

predict(
    model,
    #换自己保存的模型文件
    model_path = 'output/Atr_UNet/best_model/model.pdparams',
    transforms=transforms,
    image_list=test_list,
    save_dir='atr_result',)

2021-09-25 16:25:42 [INFO]	Loading pretrained model from output/Atr_UNet/best_model/model.pdparams
2021-09-25 16:25:42 [INFO]	There are 112/112 variables loaded into UNet.
2021-09-25 16:25:42 [INFO]	Start to predict...


400/400 [==============================] - 119s 297ms/ste


In [18]:
! mkdir Lesion_Segmentation
! mkdir Lesion_Segmentation/Atrophy/
for img in os.listdir('atr_result/pseudo_color_prediction'):
    img_dir='atr_result/pseudo_color_prediction/'+img
    im=Image.open(img_dir)
    im=Image.fromarray(np.array(im)*255)
    im.save('Lesion_Segmentation/Atrophy/'+img)

mkdir: cannot create directory ‘Lesion_Segmentation’: File exists
mkdir: cannot create directory ‘Lesion_Segmentation/Atrophy/’: File exists


## Det分割部分

由于需要Det的文件比较少，大部分文件在Det方面的分类都是不需要分割，所以仅仅使用部分样本进行训练

### 数据准备

In [4]:
! mkdir /home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Detachment_png

names=[item.split('.')[0] for item in os.listdir('work/常规赛：PALM病理性近视病灶检测与分割/Train/fundus_image')]
ori_train_list=[]
img_dir='/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/fundus_image/'
mask_dir='/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Detachment/'
mask_dir2='/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Detachment_png/'
for name in names:
    if os.path.exists(img_dir+name+'.jpg') and os.path.exists(mask_dir+name+'.bmp'):
        im=Image.open(mask_dir+name+'.bmp')
        im=(np.array(im)/255).astype('uint8')
        im = Image.fromarray(im)
        im.save(mask_dir2+name+'.png')
        ori_train_list.append(img_dir+name+'.jpg '+mask_dir2+name+'.png'+'\n')

mkdir: cannot create directory ‘/home/aistudio/work/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Detachment_png’: File exists


In [6]:
cut_percent=0.2
cut_point=int(cut_percent*len(ori_train_list))
random.shuffle(ori_train_list)

with open('det2_train.txt','w') as f:
    for item in ori_train_list[cut_point:]:
        f.write(item)

with open('det2_eval.txt','w') as f:
    for item in ori_train_list[:cut_point]:
        f.write(item)

### 训练

In [7]:
train_transforms = [
    #T.RandomHorizontalFlip(),# 水平翻转
    #T.RandomVerticalFlip(),# 垂直翻转
    #T.RandomDistort(0.4),# 随机扭曲
    #T.RandomBlur(0.3),# 高斯模糊
    #T.RandomScaleAspect(min_scale=0.5,aspect_ratio=0.5),# 随机缩放
    T.Resize(target_size=(512,512)),
    T.Normalize()  # 图像标准化
]

val_transforms = [
    T.Resize(target_size=(512,512)),
    T.Normalize()
]

In [8]:
# 构建训练集
train_dataset = Dataset(
    dataset_root='/home/aistudio',
    train_path='/home/aistudio/det2_train.txt',
    transforms=train_transforms,
    num_classes=2,
    mode='train'
)
#验证集
val_dataset = Dataset(
    dataset_root='/home/aistudio',
    val_path='/home/aistudio/det2_eval.txt',
    transforms=val_transforms,
    num_classes=2,
    mode='val'
)

In [9]:
model = paddleseg.models.UNet(num_classes=2)

In [10]:
base_lr =0.001
iters = 100

#自动调整学习率
lr =paddle.optimizer.lr.CosineAnnealingDecay(base_lr, T_max=(iters // 3), last_epoch=0.5)
u_optimizer = paddle.optimizer.Adam(lr, parameters=model.parameters())

mixtureLosses = [CrossEntropyLoss(),DiceLoss() ]
mixtureCoef = [0.1,0.9]
losses = {}
losses['types'] = [MixedLoss(mixtureLosses, mixtureCoef)]
losses['coef'] = [1]

In [11]:
#进行训练
train(
    model = model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=u_optimizer,
    save_dir='output/Det_UNet',
    iters=iters,  
    batch_size=8, 
    save_interval=20,
    log_iters=20,
    num_workers=2,
    losses=losses,
    # losses=CrossEntropyLoss(),
    use_vdl=False
    )

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.int64, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


2021-09-25 20:44:50 [INFO]	[TRAIN] epoch: 20, iter: 20/100, loss: 0.2679, lr: 0.000314, batch_cost: 2.0074, reader_cost: 1.51707, ips: 3.9853 samples/sec | ETA 00:02:40
2021-09-25 20:44:50 [INFO]	Start evaluating (total_samples: 3, total_iters: 3)...


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int32, but right dtype is paddle.bool, the right dtype will convert to paddle.int32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.bool, the right dtype will convert to paddle.int64
  format(lhs_dtype, rhs_dtype, lhs_dtype))
3/3 [==============================] - 1s 209ms/step - batch_cost: 0.2080 - reader cost: 0.18


2021-09-25 20:44:51 [INFO]	[EVAL] #Images: 3 mIoU: 0.6211 Acc: 0.7716 Kappa: 0.5430 
2021-09-25 20:44:51 [INFO]	[EVAL] Class IoU: 
[0.6726 0.5696]
2021-09-25 20:44:51 [INFO]	[EVAL] Class Acc: 
[0.704  0.9066]
2021-09-25 20:44:52 [INFO]	[EVAL] The model with the best validation mIoU (0.6211) was saved at iter 20.
2021-09-25 20:45:32 [INFO]	[TRAIN] epoch: 40, iter: 40/100, loss: 0.1976, lr: 0.000122, batch_cost: 2.0017, reader_cost: 1.51195, ips: 3.9966 samples/sec | ETA 00:02:00
2021-09-25 20:45:32 [INFO]	Start evaluating (total_samples: 3, total_iters: 3)...


3/3 [==============================] - 1s 212ms/step - batch_cost: 0.2107 - reader cost: 0.19


...

3/3 [==============================] - 1s 211ms/step - batch_cost: 0.2101 - reader cost: 0.19


2021-09-25 20:46:56 [INFO]	[EVAL] #Images: 3 mIoU: 0.6044 Acc: 0.7557 Kappa: 0.5113 
2021-09-25 20:46:56 [INFO]	[EVAL] Class IoU: 
[0.6386 0.5701]
2021-09-25 20:46:56 [INFO]	[EVAL] Class Acc: 
[0.7108 0.8252]
2021-09-25 20:46:57 [INFO]	[EVAL] The model with the best validation mIoU (0.6468) was saved at iter 40.
2021-09-25 20:47:37 [INFO]	[TRAIN] epoch: 100, iter: 100/100, loss: 0.1575, lr: 0.000005, batch_cost: 2.0311, reader_cost: 1.54091, ips: 3.9387 samples/sec | ETA 00:00:00
2021-09-25 20:47:37 [INFO]	Start evaluating (total_samples: 3, total_iters: 3)...


3/3 [==============================] - 1s 206ms/step - batch_cost: 0.2048 - reader cost: 0.18


2021-09-25 20:47:38 [INFO]	[EVAL] #Images: 3 mIoU: 0.6517 Acc: 0.7904 Kappa: 0.5808 
2021-09-25 20:47:38 [INFO]	[EVAL] Class IoU: 
[0.6766 0.6268]
2021-09-25 20:47:38 [INFO]	[EVAL] Class Acc: 
[0.7481 0.8505]
2021-09-25 20:47:39 [INFO]	[EVAL] The model with the best validation mIoU (0.6517) was saved at iter 100.
<class 'paddle.nn.layer.conv.Conv2D'>'s flops has been counted
<class 'paddle.nn.layer.norm.BatchNorm2D'>'s flops has been counted
Cannot find suitable count function for <class 'paddle.nn.layer.pooling.MaxPool2D'>. Treat it as zero FLOPs.
Total Flops: -1150812160     Total Params: 13404354


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


### 进行预测，保存结果到指定目录

In [12]:
test_list=["work/常规赛：PALM病理性近视病灶检测与分割/PALM-Testing400-Images/"+item for item in os.listdir("work/常规赛：PALM病理性近视病灶检测与分割/PALM-Testing400-Images")]

transforms = T.Compose([
    T.Resize(target_size=(512, 512)),
    T.Normalize()
])

model = paddleseg.models.UNet(num_classes=2)

predict(
    model,
    #换自己保存的模型文件
    model_path = 'output/Det_UNet/best_model/model.pdparams',
    transforms=transforms,
    image_list=test_list,
    save_dir='det_result',)

2021-09-25 20:50:21 [INFO]	Loading pretrained model from output/Det_UNet/best_model/model.pdparams
2021-09-25 20:50:21 [INFO]	There are 112/112 variables loaded into UNet.
2021-09-25 20:50:21 [INFO]	Start to predict...


400/400 [==============================] - 121s 303ms/st


In [14]:
! mkdir Lesion_Segmentation
! mkdir Lesion_Segmentation/Detachment/
for img in os.listdir('det_result/pseudo_color_prediction'):
    img_dir='det_result/pseudo_color_prediction/'+img
    im=Image.open(img_dir)
    im=Image.fromarray(np.array(im)*255)
    im.save('Lesion_Segmentation/Detachment/'+img)

mkdir: cannot create directory ‘Lesion_Segmentation’: File exists
mkdir: cannot create directory ‘Lesion_Segmentation/Detachment/’: File exists


## 打包结果

In [15]:
! zip -q -r result.zip Lesion_Segmentation

# 总结

如果直接训练分割模型，模型几乎将所有的样本都判定为需要分割，从而使得最终评价的分类得分(F1)极差，因此可以考虑加入分类模型，对分类模型的结果中需要分割的再进行分割。根据我在十月份的测试，这种先分类再分割的方法可以显著提高评价中F1指标的分数。
![](https://ai-studio-static-online.cdn.bcebos.com/2066e8fba6014a388aa8b702d044ea90cf014012e3874df4b58232d0c83ac22a)

但是这种方法同样会由于部分正例没有被分割到从而降低Dice的分数，并不会使得整体得分大幅度提高。但是如果想要让整体评价提高，那么每一项都有必要取得一个比较好的得分的！所以在基本的分类基础上做亿点点的调优操作(包含了写出了bug误打误撞等操作)之后是可以达到一个比较令人满意的分数的(比如0.8+的分数)！感兴趣的朋友们可以尝试一下这种**先分类再分割**的框架。最后附上一个因为bug产生的0.8+的结果~
![](https://ai-studio-static-online.cdn.bcebos.com/8dc33367fb4b4ae387ce030adf35bba623dc60ceeb194a4496e6b07d93969d9e)


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 